## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [1]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [5]:
import tensorflow as tf
print(tf.__version__)

2.0.0


### Collect Data

In [0]:
from google.colab import drive
drive.mount('/gdrive')

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import pandas as pd

In [0]:
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Shared/Residency 6/prices.csv')

### Check all columns in the dataset

In [6]:
data.shape

(851264, 7)

In [7]:
data.columns

Index(['date', 'symbol', 'open', 'close', 'low', 'high', 'volume'], dtype='object')

In [8]:
data.head()

,date,symbol,open,close,low,high,volume
0,2016-01-05 00:00:00,WLTW,123.430000,125.839996,122.309998,126.250000,2163600.0
1,2016-01-06 00:00:00,WLTW,125.239998,119.980003,119.940002,125.540001,2386400.0
2,2016-01-07 00:00:00,WLTW,116.379997,114.949997,114.930000,119.739998,2489500.0
3,2016-01-08 00:00:00,WLTW,115.480003,116.620003,113.500000,117.440002,2006300.0
4,2016-01-11 00:00:00,WLTW,117.010002,114.970001,114.089996,117.330002,1408600.0


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851264 entries, 0 to 851263
Data columns (total 7 columns):
date      851264 non-null object
symbol    851264 non-null object
open      851264 non-null float64
close     851264 non-null float64
low       851264 non-null float64
high      851264 non-null float64
volume    851264 non-null float64
dtypes: float64(5), object(2)
memory usage: 45.5+ MB


In [10]:
data.describe()

,open,close,low,high,volume
count,851264.000000,851264.000000,851264.000000,851264.000000,8.512640e+05
mean,70.836986,70.857109,70.118414,71.543476,5.415113e+06
std,83.695876,83.689686,82.877294,84.465504,1.249468e+07
min,0.850000,0.860000,0.830000,0.880000,0.000000e+00
25%,33.840000,33.849998,33.480000,34.189999,1.221500e+06
50%,52.770000,52.799999,52.230000,53.310001,2.476250e+06
75%,79.879997,79.889999,79.110001,80.610001,5.222500e+06
max,1584.439941,1578.130005,1549.939941,1600.930054,8.596434e+08


### Drop columns `date` and  `symbol`

In [0]:
data.drop(['date', 'symbol'], axis=1, inplace=True)

In [12]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [0]:
first_1000_data = data[:1000]

In [0]:
first_1000_data['volume'] = (first_1000_data['volume'] / 1000000)

In [39]:
first_1000_data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2.1636
1,125.239998,119.980003,119.940002,125.540001,2.3864
2,116.379997,114.949997,114.930000,119.739998,2.4895
3,115.480003,116.620003,113.500000,117.440002,2.0063
4,117.010002,114.970001,114.089996,117.330002,1.4086


### Divide the data into train and test sets

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
temp_data = data
temp_data['volume'] = (temp_data['volume'] / 1000000)

In [0]:
X = first_1000_data.drop('volume', axis=1, inplace=False)

y = first_1000_data[['volume']]

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=4)

#### Convert Training and Test Data to numpy float32 arrays


In [0]:
X_train = tf.cast(X_train.values, tf.float32)
X_test = tf.cast(X_test.values, tf.float32)
y_train = tf.cast(y_train.values, tf.float32)
y_test = tf.cast(y_test.values, tf.float32)

In [0]:
# X_train = X_train.astype('float32')
# X_test = X_test.astype('float32')
# y_train = y_train.astype('float32')
# y_test = y_test.astype('float32')

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [0]:
from sklearn.preprocessing import Normalizer

normalizer_model = Normalizer()

In [0]:
normalizer_model.fit(X_train)
scaled_X_train = normalizer_model.transform(X_train)
scaled_X_test = normalizer_model.transform(X_test)

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [0]:
#  We are initializing weights and Bias of (Input-Hidden Layer)
W = tf.zeros(shape=(4, 1))
b = tf.zeros(shape=(1))

In [0]:
# #We are initializing weights and Bias of (Hidden-Output)
# w2 = tf.random_normal(shape=(5,1))
# b2 = tf.zeros(shape=(1))

2.Define a function to calculate prediction

In [0]:
@tf.function
def prediction(features, W, b):
    y_pred = tf.add(tf.matmul(features, W), b)
    return y_pred

3.Loss (Cost) Function [Mean square error]

In [0]:
@tf.function
def loss(y_actual, y_predicted):
    diff = y_actual - y_predicted
    sqr = tf.square(diff)
    avg = tf.reduce_mean(sqr)
    return avg

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [0]:
@tf.function
def train(x, y_actual, w, b, learning_rate=0.01):
    
    # Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        t.watch([w,b])
        current_prediction = prediction(x, w, b)
        current_loss = loss(y_actual, current_prediction)
    
    # Calculate Gradients for Loss with respect to Weights and Bias
    dw, db = t.gradient(current_loss,[w, b])
    
    # Update Weights and Bias
    w = w - learning_rate * dw
    b = b - learning_rate * db
    
    return w, b

## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [46]:
for i in range(100):    
    W, b = train(scaled_X_train, y_train, W, b)
    if(i%5 == 0):
        print('Current Training Loss on iteration', i, loss(y_train, prediction(scaled_X_train, W, b)).numpy())    

Current Training Loss on iteration 0 219.92038
Current Training Loss on iteration 5 211.47873
Current Training Loss on iteration 10 205.86636
Current Training Loss on iteration 15 202.135
Current Training Loss on iteration 20 199.65427
Current Training Loss on iteration 25 198.00497
Current Training Loss on iteration 30 196.90842
Current Training Loss on iteration 35 196.1794
Current Training Loss on iteration 40 195.69464
Current Training Loss on iteration 45 195.3724
Current Training Loss on iteration 50 195.1581
Current Training Loss on iteration 55 195.01569
Current Training Loss on iteration 60 194.92094
Current Training Loss on iteration 65 194.85794
Current Training Loss on iteration 70 194.81607
Current Training Loss on iteration 75 194.78816
Current Training Loss on iteration 80 194.76962
Current Training Loss on iteration 85 194.75728
Current Training Loss on iteration 90 194.74908
Current Training Loss on iteration 95 194.74362


### Get the shapes and values of W and b

In [47]:
W

<tf.Tensor: id=963, shape=(4, 1), dtype=float32, numpy=
array([[1.265348 ],
       [1.3042523],
       [1.2596458],
       [1.3096504]], dtype=float32)>

In [48]:
b

<tf.Tensor: id=964, shape=(1,), dtype=float32, numpy=array([2.5699263], dtype=float32)>

### Model Prediction on 1st Examples in Test Dataset

In [49]:
pred = prediction(scaled_X_test, W, b)
print("Loss : {:.3f}".format(loss(pred, y_test)))

Loss : 242.525


## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [0]:
iris_data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Shared/Residency 6/11_Iris.csv')

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [51]:
iris_data.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


### Splitting the data into feature set and target set

In [0]:
X = iris_data.drop(['Id', 'Species'], axis=1, inplace=False)
y = iris_data[['Species']] 
# y = pd.get_dummies(y)

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=4)

In [98]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(y)
y = le.transform(y)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=4)

y_train = tf.keras.utils.to_categorical(y_train, num_classes=3)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=3)

###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [0]:
# Initialize Sequential model
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Dense(3, input_dim=4, kernel_initializer='normal', activation='relu'))

# Add Dense Layer which provides 10 Outputs after applying softmax
model.add(tf.keras.layers.Dense(3, activation='softmax'))

In [0]:
# Comile the model
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy', tf.keras.metrics.CategoricalAccuracy()])

### Model Training 

In [103]:
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50,
          batch_size = X_train.shape[0])

Train on 105 samples, validate on 45 samples
Epoch 1/50
105/105 [==============================] - 0s 982us/sample - loss: 1.0470 - accuracy: 0.3429 - categorical_accuracy: 0.3429 - val_loss: 1.0516 - val_accuracy: 0.3111 - val_categorical_accuracy: 0.3111
Epoch 2/50
105/105 [==============================] - 0s 80us/sample - loss: 1.0367 - accuracy: 0.3429 - categorical_accuracy: 0.3429 - val_loss: 1.0462 - val_accuracy: 0.3111 - val_categorical_accuracy: 0.3111
Epoch 3/50
105/105 [==============================] - 0s 63us/sample - loss: 1.0275 - accuracy: 0.3429 - categorical_accuracy: 0.3429 - val_loss: 1.0400 - val_accuracy: 0.3111 - val_categorical_accuracy: 0.3111
Epoch 4/50
105/105 [==============================] - 0s 65us/sample - loss: 1.0189 - accuracy: 0.3429 - categorical_accuracy: 0.3429 - val_loss: 1.0331 - val_accuracy: 0.3111 - val_categorical_accuracy: 0.3111
Epoch 5/50
105/105 [==============================] - 0s 66us/sample - loss: 1.0108 - accuracy: 0.3429 - categ

### Model Prediction

In [87]:
model.predict(X_test)

array([[0.23808855, 0.47041428, 0.2914972 ],
       [0.36020252, 0.31986544, 0.31993204],
       [0.22749622, 0.48880926, 0.28369457],
       [0.22100109, 0.50024074, 0.2787581 ],
       [0.22706187, 0.48957008, 0.28336808],
       [0.2649335 , 0.43046468, 0.30460188],
       [0.24877416, 0.4521659 , 0.29906   ],
       [0.37450662, 0.30677074, 0.31872267],
       [0.3610263 , 0.31730995, 0.32166374],
       [0.24985659, 0.45121968, 0.29892376],
       [0.34786275, 0.32993194, 0.32220528],
       [0.3555544 , 0.32031196, 0.32413357],
       [0.35372683, 0.32271403, 0.3235591 ],
       [0.26136938, 0.43541622, 0.30321437],
       [0.24682009, 0.45547986, 0.29770002],
       [0.35324115, 0.3236212 , 0.32313764],
       [0.27618775, 0.41220433, 0.31160793],
       [0.3590295 , 0.31647763, 0.32449287],
       [0.3481762 , 0.32796717, 0.32385662],
       [0.2497852 , 0.45154715, 0.2986676 ],
       [0.36162657, 0.31543905, 0.3229344 ],
       [0.24129343, 0.4649087 , 0.29379788],
       [0.

In [105]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_14 (Dense)             (None, 3)                 15        
_________________________________________________________________
dense_15 (Dense)             (None, 3)                 12        
Total params: 27
Trainable params: 27
Non-trainable params: 0
_________________________________________________________________


In [109]:
import numpy as np
y_pred = model.predict(X_test)

y_test_class = np.argmax(y_test,axis=1)
y_pred_class = np.argmax(y_pred,axis=1)

In [113]:
from sklearn.metrics import classification_report,confusion_matrix
print(classification_report(y_pred_class,y_test_class))
print(confusion_matrix(y_pred_class,y_test_class))

              precision    recall  f1-score   support

           0       0.62      1.00      0.76        13
           1       0.00      0.00      0.00         8
           2       1.00      0.58      0.74        24

    accuracy                           0.60        45
   macro avg       0.54      0.53      0.50        45
weighted avg       0.71      0.60      0.61        45

[[13  0  0]
 [ 8  0  0]
 [ 0 10 14]]


In [116]:
model.evaluate(X_test,y_test)

45/45 [==============================] - 0s 289us/sample - loss: 0.8999 - accuracy: 0.6000 - categorical_accuracy: 0.6000


[0.89992782274882, 0.6, 0.6]

### Save the Model

In [117]:
import pickle
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))

TypeError: ignored

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?